In [92]:
import requests

url = "http://127.0.0.1:8000/api/ingest"
file = {"file": open("data/window.jpg", "rb")}
resp = requests.post(url=url, files=file)
resp

<Response [200]>

In [93]:
list_of_values = resp.json()
for value in list_of_values:
    print(value)

image
comment
date
latitude
longitude
altitude
location
direction
ai_comment
ai_comment_vector


In [94]:
list_of_values['ai_comment']

"The image shows an interior view looking out of a window. It appears to be an overcast day, and the view outside is partially obscured by a tall, light-colored wall on the right side which limits the perspective of the outdoors. On the left outside the window, you can see the rooftop of a building with multiple dormer windows, suggesting an urban residential area.\n\nInside, on the windowsill, there is a blue-colored, slim, rectangular object which appears to be a smartphone lying face-down. Below the windowsill, there's a heater or radiator attached to the wall. The floor is covered with what looks like wooden laminate flooring, and on the extreme bottom left of the image, there's a tiny portion of what might be another object, perhaps a piece of furniture but it's too little to identify clearly."